In [1]:
# load DF
import pandas as pd
df = pd.read_pickle('/workspace/data/rev-moenk/training/df_2022-04-10 13:00:01.pkl')

In [49]:
# add v_x and v_y columns
import datetime
timedelta = datetime.timedelta(seconds=1)



df['v_x'] = -1* df.groupby(['mmsi'])['x'].diff(-1).fillna(-1* df['v_x'].shift(1)) / timedelta.total_seconds()
df['v_y'] = -1* df.groupby(['mmsi'])['y'].diff(-1).fillna(-1* df['v_y'].shift(1)) / timedelta.total_seconds()

In [55]:
# calculate delta_v = norm(v_x, v_y).shift(1) - norm(v_x, v_y) 
import numpy as np
df['delta_v'] = np.sqrt(df['v_x']**2 + df['v_y']**2).shift(-1, fill_value=0) - np.sqrt(df['v_x']**2 + df['v_y']**2)
# its okay to leave the last row in each group as it will not be used for training as we dont have target values anyway



In [ ]:


def perpendicular_vector(v):
    if v[1] == 0:
        if v[0] == 0:
            raise ValueError('zero vector')
        else:
            return np.cross(v, [0, 1])
    return np.cross(v, [1, 0])

# calculate delta_angle = (∆pt+1 − ∆pt)T (∆pt)⊥/‖∆pt‖2

df['delta_angle'] = np.transpose([df['v_x'].shift(1) - df['v_x'], df['v_y'].shift(1) - df['v_y']]) * np.cross([df['v_x'],df['v_y']], [1,0]) / np.linalg.norm([df['v_x'],df['v_y']])
